D-wave classes

In [1]:
import dimod #interface for D-wave
import neal  #simulator

Scientific python libraries

In [5]:
import numpy as np              #scientific library
from numpy import linalg as LA  #linear algebra package

Input

In [27]:
dim_matrix  =8       #dimension of the matrix  
num_of_bits =2       #number of bits used for diagonalization
num_of_reads=50      #number of reads from the simulator
zero_energy =False   #first part of the algorithm you use only the zero energy (True) or the full response (False) 
beta        =100     #beta used for weighting the full response
epsilon_tol =0.0001  #tollerance (the precision of the eigenvlue)

Precision vector and precision matrix 

In [28]:
p=np.empty([num_of_bits])
p[0]=-1
for b in range(2,num_of_bits+1):
    p[b-1]=2**(-b+1)
P=np.empty([dim_matrix,dim_matrix*num_of_bits])
P[:][:]=0
for i in range(dim_matrix):
    P[i][i*num_of_bits:(i+1)*num_of_bits]=p[:]

Function for writing arrays and vectors in the binary basis

In [29]:
def bin_matrix(A,prec):                       #prec=precision-->needed in the second part of the algorithm
    QA=prec**2*np.matmul(P.T,np.matmul(A,P))
    return QA
def bin_vector(v,prec):
    Qv=prec*np.matmul(v.T,P)
    return Qv

This comand select the anealer (simulator in this case)

In [30]:
sampler = neal.SimulatedAnnealingSampler()

Here we define the matrix to diagonalize and we compute for test the first eigenvector and first eigenvalue

In [31]:
#definition of the matrix (random)
I=np.eye(dim_matrix,dim_matrix)
np.random.seed(1234)
A=10*(np.random.rand(dim_matrix,dim_matrix)-0.5)
for i in range(dim_matrix):
    for j in range(dim_matrix):
        A[j][i]=A[i][j]
#eigenvalue and eigenvector computation
eval,evec=LA.eigh(A)
evec0=evec[::,0]
print("First eignevalue = ",eval[0])
print("First eigenvector= ",evec0)

First eignevalue =  -11.661330729503852
First eigenvector=  [-5.46368074e-01 -1.59875279e-05 -1.76301666e-01  3.37448410e-01
  2.03154093e-01 -3.83234216e-01 -4.91878740e-01  3.55588915e-01]


First part of the algorithm: initial guess phase

In [32]:
lam=10**6
tmp_lam=np.trace(A)/dim_matrix
print("Iteration= ",0,"Starting eigenvalue =", tmp_lam)
j=0
while tmp_lam < lam:
    j=j+1
    lam=tmp_lam
    #Rewriting the matrix in binary form
    QA=bin_matrix(A-lam*I,1) 
    #Transform our matrix in one managable by the D-wave                                                                                                              
    A_bqm = dimod.BinaryQuadraticModel.from_qubo(QA,offset=0.0)
    #Sampling                                                                                                                                                         
    sampleset = sampler.sample(A_bqm,num_reads=num_of_reads,beta_range=[0.1, 4.2],seed=3456)
    #Sample database                                                                                                                                                  
    rsample=sampleset.record
    #selecting the state with energy zero                                                                                                                                                   
    energy0=10**6
    for i in range(num_of_reads):
        energy=rsample[i][1]
        if(energy<energy0):
            energy0=energy
            indx=i
    if(zero_energy):                  #only state with zero energy is selected                                                                                     
        x=np.array(rsample[indx][0])
    else:                             #linear comination of the full response
        E0=rsample[indx][1]
        x=np.empty([dim_matrix*num_of_bits])
        for i in range(num_of_reads):
            x=x+np.exp(-beta*(rsample[i][1]-E0))*rsample[i][0]
        x=x/num_of_reads
    if(np.matmul(x.T,x)<10**(-16)):   #if the best eigenvector is zero exit the loop
        print("exit loop")
        break
    tmp_v=np.matmul(P,x)                           #new eigenvector
    tmp_v_norm=np.sqrt(np.matmul(tmp_v.T,tmp_v))
    tmp_v=tmp_v/tmp_v_norm
    tmp_lam=np.matmul(tmp_v.T,np.matmul(A,tmp_v))  #new eigenvalue
    if(tmp_lam < lam):
        v=tmp_v
    print("Iteration= ",j," old value= ",lam," new value= ",tmp_lam)
print("End of the initial guess part")
print("Eigenvalue = ",lam)
print("Eigenvector= ",v)

Iteration=  0 Starting eigenvalue = 0.24278500727142616
Iteration=  1  old value=  0.24278500727142616  new value=  -9.527464976225776
Iteration=  2  old value=  -9.527464976225776  new value=  -10.261918341124277
Iteration=  3  old value=  -10.261918341124277  new value=  -9.794475195863637
End of the initial guess part
Eigenvalue =  -10.261918341124277
Eigenvector=  [-0.53131947 -0.06248993 -0.27589133  0.16409756  0.18347507 -0.53376626
 -0.50989296  0.1794164 ]


Iterative desendent phase

In [33]:
j=0
precision=0.1                     #precision
while precision > epsilon_tol:                                                                                                                                                          
    j=j+1
    #Preparing the descendent direction
    H=A-lam*I
    vH=2.0*np.matmul(v.T,H)    
    QH=bin_matrix(H,precision)
    QvH=bin_vector(vH,precision)
    for i in range(dim_matrix*num_of_bits):
        QH[i][i]=QH[i][i]+QvH[i]
    #Transform our matrix in one managable by the D-wave                                                                                                              
    A_bqm = dimod.BinaryQuadraticModel.from_qubo(QH,offset=0.0)
    #Sampling                                                                                                                                                         
    sampleset = sampler.sample(A_bqm,num_reads=num_of_reads,beta_range=[0.1, 4.2],seed=3456)
    #Sample database                                                                                                                                                  
    rsample=sampleset.record
    #Select the zero energy state                                                                                                                                                   
    energy0=10**6
    for i in range(num_of_reads):
        energy=rsample[i][1]
        if(energy<energy0):
            energy0=energy
            indx=i                                                
    x=np.array(rsample[indx][0])
    delta=precision*np.matmul(P,x)       #delta vector
    delta=delta-np.matmul(delta.T,v)*v   #selcting the orthogonal part
    #Line search step
    Hd=np.matmul(H,delta)                                                                                                                          
    if(np.matmul(delta.T,delta)<10**(-16)):                                                                                                                              
        precision=precision*0.1
        print("Iteration= ",j,"new precision = ",precision)
        continue
    tmin=-np.matmul(v.T,Hd)/np.matmul(delta,Hd)
    tmin=max(tmin,1)                                                                                                                                     
    delta=delta*tmin
    #new eigenvector
    tmp_v=v+delta
    tmp_norm=np.sqrt(np.matmul(tmp_v.T,tmp_v))
    tmp_v=tmp_v/tmp_norm
    #new eigenvalue
    tmp_lam=np.matmul(tmp_v.T,np.matmul(A,tmp_v))
    if(tmp_lam<lam):
        v=tmp_v
        print("Iteration= ",j," old value= ",lam," new value= ",tmp_lam)
        lam=tmp_lam
    else:
        precision=precision*0.1
        print("Iteration= ",j,"new precision = ",precision)
print("Final values")
print("Eigenvalue = ",lam)
print("Eigenvector= ",v)

Iteration=  1  old value=  -10.261918341124277  new value=  -11.366148483626661
Iteration=  2  old value=  -11.366148483626661  new value=  -11.551359042261947
Iteration=  3 new precision =  0.010000000000000002
Iteration=  4  old value=  -11.551359042261947  new value=  -11.578843391217667
Iteration=  5  old value=  -11.578843391217667  new value=  -11.593959210494736
Iteration=  6  old value=  -11.593959210494736  new value=  -11.608072973446747
Iteration=  7  old value=  -11.608072973446747  new value=  -11.625170708509806
Iteration=  8  old value=  -11.625170708509806  new value=  -11.639367035563081
Iteration=  9  old value=  -11.639367035563081  new value=  -11.644337585787222
Iteration=  10  old value=  -11.644337585787222  new value=  -11.650649785507722
Iteration=  11  old value=  -11.650649785507722  new value=  -11.65655407030124
Iteration=  12  old value=  -11.65655407030124  new value=  -11.657420543800416
Iteration=  13  old value=  -11.657420543800416  new value=  -11.65

Iteration=  104  old value=  -11.66115256021787  new value=  -11.661153249620668
Iteration=  105  old value=  -11.661153249620668  new value=  -11.661154223736137
Iteration=  106  old value=  -11.661154223736137  new value=  -11.661155066364218
Iteration=  107  old value=  -11.661155066364218  new value=  -11.661155989136176
Iteration=  108  old value=  -11.661155989136176  new value=  -11.661156785898951
Iteration=  109  old value=  -11.661156785898951  new value=  -11.661157761433422
Iteration=  110  old value=  -11.661157761433422  new value=  -11.661158661193479
Iteration=  111  old value=  -11.661158661193479  new value=  -11.661159882912125
Iteration=  112  old value=  -11.661159882912125  new value=  -11.661161385306041
Iteration=  113  old value=  -11.661161385306041  new value=  -11.661162233316047
Iteration=  114  old value=  -11.661162233316047  new value=  -11.661162944755942
Iteration=  115  old value=  -11.661162944755942  new value=  -11.661163712840487
Iteration=  116  

Iteration=  206  old value=  -11.661229635517435  new value=  -11.661230151101417
Iteration=  207  old value=  -11.661230151101417  new value=  -11.661230524963349
Iteration=  208  old value=  -11.661230524963349  new value=  -11.661231103265795
Iteration=  209  old value=  -11.661231103265795  new value=  -11.66123155860367
Iteration=  210  old value=  -11.66123155860367  new value=  -11.661232417881765
Iteration=  211  old value=  -11.661232417881765  new value=  -11.661232847110961
Iteration=  212  old value=  -11.661232847110961  new value=  -11.661233419107104
Iteration=  213  old value=  -11.661233419107104  new value=  -11.661234169186235
Iteration=  214  old value=  -11.661234169186235  new value=  -11.661234568161822
Iteration=  215  old value=  -11.661234568161822  new value=  -11.661235185036134
Iteration=  216  old value=  -11.661235185036134  new value=  -11.661235677332392
Iteration=  217  old value=  -11.661235677332392  new value=  -11.661236091302934
Iteration=  218  o

Iteration=  310  old value=  -11.661274844715598  new value=  -11.661275026263032
Iteration=  311  old value=  -11.661275026263032  new value=  -11.661275389051088
Iteration=  312  old value=  -11.661275389051088  new value=  -11.661275932246618
Iteration=  313  old value=  -11.661275932246618  new value=  -11.66127623767284
Iteration=  314  old value=  -11.66127623767284  new value=  -11.661276530364939
Iteration=  315  old value=  -11.661276530364939  new value=  -11.66127701177178
Iteration=  316  old value=  -11.66127701177178  new value=  -11.661277336687984
Iteration=  317  old value=  -11.661277336687984  new value=  -11.661277611556354
Iteration=  318  old value=  -11.661277611556354  new value=  -11.66127808284333
Iteration=  319  old value=  -11.66127808284333  new value=  -11.661278285075456
Iteration=  320  old value=  -11.661278285075456  new value=  -11.661278670916921
Iteration=  321  old value=  -11.661278670916921  new value=  -11.661278976142452
Iteration=  322  old v

Iteration=  412  old value=  -11.66130285418263  new value=  -11.66130313170406
Iteration=  413  old value=  -11.66130313170406  new value=  -11.661303143218042
Iteration=  414  old value=  -11.661303143218042  new value=  -11.661303400683572
Iteration=  415  old value=  -11.661303400683572  new value=  -11.661303865131641
Iteration=  416  old value=  -11.661303865131641  new value=  -11.661304036557997
Iteration=  417  old value=  -11.661304036557997  new value=  -11.661304107952079
Iteration=  418  old value=  -11.661304107952079  new value=  -11.661304361664275
Iteration=  419  old value=  -11.661304361664275  new value=  -11.661304650226603
Iteration=  420  old value=  -11.661304650226603  new value=  -11.661304866498053
Iteration=  421  old value=  -11.661304866498053  new value=  -11.661304913448717
Iteration=  422  old value=  -11.661304913448717  new value=  -11.661305168945184
Iteration=  423  old value=  -11.661305168945184  new value=  -11.661305345122141
Iteration=  424  ol